In [1]:
import tensorflow as tf
from tensorflow import keras
import h5py
import scipy.stats
import numpy as np
import os
import pandas as pd
from sklearn import model_selection
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
celltype = 'HepG2'

2023-09-27 15:53:20.549816: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#model = keras.models.load_model('/home/ztang/multitask_RNA/model_rep/MPRAnn/'+celltype+'/model_0.h5')
model = keras.models.load_model('/home/ztang/multitask_RNA/model/lenti_MPRA_onehot/'+celltype+'/ResNet.h5')

2023-09-27 15:53:23.508006: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78973 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:85:00.0, compute capability: 8.0
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 230, 4)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 230, 196)     11956       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 230, 196)    784         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv_activation (Activation)   (None, 230, 196)     0           ['batch_normalization[0][0]']

In [4]:
f = h5py.File('/home/ztang/multitask_RNA/data/lenti_MPRA/'+celltype+'_onehot.h5', 'r')
x = f['onehot'][()]
y = f['target'][()]
x = np.swapaxes(x,1,2)
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.1,random_state=42)
model.evaluate(x_test,y_test)

2023-09-27 15:53:26.234961: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800


 81/438 [====>.........................] - ETA: 0s - loss: 0.2797 - mse: 0.2797 - mae: 0.3903 

2023-09-27 15:53:26.775657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


438/438 [==============================] - 2s 2ms/step - loss: 0.2749 - mse: 0.2749 - mae: 0.3891


[0.2749405801296234, 0.2749405801296234, 0.38910195231437683]

In [5]:
## Test set performance
y_pred = model.predict(x_test)
scipy.stats.pearsonr(np.squeeze(y_pred),y_test)

438/438 [==============================] - 1s 1ms/step


PearsonRResult(statistic=0.7467120731312961, pvalue=0.0)

In [9]:
file = h5py.File("/home/ztang/multitask_RNA/data/CAGI/"+celltype+"/onehot.h5", "r")
alt = file['alt']
ref = file['ref']
alt_pred = model.predict(alt)
ref_pred = model.predict(ref)
pred = alt_pred - ref_pred
pred.shape

114/114 [==============================] - 0s 1ms/step


(3647, 1)

In [13]:
exp_df = pd.read_csv('/home/ztang/multitask_RNA/data/CAGI/'+celltype+'/metadata.csv')
target = exp_df['6'].values.tolist()

In [8]:
#CAGI performance
start_idx=0
perf = []
for exp in exp_df['8'].unique():
    sub_df = exp_df[exp_df['8'] == exp]
    exp_target = np.array(target)[sub_df.index.to_list()]
    exp_pred = np.squeeze(pred)[sub_df.index.to_list()]
    print(exp)
    perf.append(scipy.stats.pearsonr(exp_pred,exp_target)[0])
    print(scipy.stats.pearsonr(exp_pred,exp_target)[0])

LDLR
0.5142324165067679
SORT1
0.4413440653055404
F9
0.5024509375518791


## All CAGI performance

In [ ]:
import tensorflow as tf
from tensorflow import keras
import h5py
import scipy.stats
import numpy as np
import os
import pandas as pd
from sklearn import model_selection
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
celltype = 'HepG2'

In [ ]:
model = keras.models.load_model('/home/ztang/multitask_RNA/model/lenti_MPRA_onehot/'+celltype+'/model.h5')

In [ ]:
file = h5py.File("/home/ztang/multitask_RNA/data/CAGI/230/CAGI_onehot.h5", "r")
alt = file['alt']
ref = file['ref']
alt_pred = model.predict(alt)
ref_pred = model.predict(ref)
pred = alt_pred - ref_pred
pred.shape

In [ ]:
cagi_df = pd.read_csv('../../data/CAGI/230/final_cagi_metadata.csv',
                      index_col=0).reset_index()
exp_list = cagi_df['8'].unique()
target = cagi_df['6'].values.tolist()

In [ ]:
import scipy.stats as stats
perf = []
sanity_check = 0
for exp in cagi_df['8'].unique():
    sub_df = cagi_df[cagi_df['8'] == exp]
    sanity_check += len(sub_df)
    exp_target = np.array(target)[sub_df.index.to_list()]
    exp_pred = np.squeeze(pred)[sub_df.index.to_list()]
    print(exp)
    perf.append(stats.pearsonr(exp_pred,exp_target)[0])
    print(stats.pearsonr(exp_pred,exp_target)[0])

In [ ]:
np.mean(perf)